In [3]:
# import data manipulation tools
import numpy as np
import pandas as pd
# import file access tools
import os
import env

In [2]:
def acquire_web_traffic():
    '''
    this will read in the codeup web traffic log from csv file in the local directory,
    if one does not exist, it will download the data via sql from the codeup server
    '''
    # set file name
    filename = 'curriculum_logs.csv'
    # check if file exists in local directory
    if os.path.exists(filename):
        print('opening file from local directory')
        # read in the file to a dataframe
        traffic = pd.read_csv(filename, index_col=0)
    # if the file doesn't exist
    else:
        # print an error
        print(f'file {filename} not found in local directory, downloading via SQL')
        # download dataset from codeup server
        # set database we are looking at
        database = 'curriculum_logs'
        # create sql connectioni
        connection = env.get_db_url(database)
        # create sql query
        query = '''
select l.date, l.time, l.path, l.user_id, l.cohort_id,
	l.ip, c.name as cohort_name, c.start_date, c.end_date, c.program_id
from logs l
left join cohorts c
	on l.cohort_id = c.id
;
        '''
        # query server via sql
        traffic = pd.read_sql(query, connection)
        # cache the data to local csv file
        traffic.to_csv(filename)
    # return the dataframe
    return traffic

In [22]:
def prepare_web_traffic(traffic):
    '''
    this will prepare the codeup web traffic dataframe for use
    '''
    # combine the date and time into one varaible
    traffic['datetime'] = traffic.date + ' ' + traffic.time
    # change the new datetime column to datetime type
    traffic.datetime = pd.to_datetime(traffic.datetime)
    # drop the redundant date and time columns and set datetime as the index
    traffic = traffic.drop(columns=['date', 'time']).set_index('datetime')
    # drop null rows in path (there's only 1)
    traffic = traffic[traffic.path.isna() == False]
    # convert first part of website path into column called lesson
    # create an empty list
    page_list=[]
    # split the webpage path by '/'
    pages = traffic.path.str.split('/')
    # go through all of the pages
    for i in range(len(pages)):
        # make a list of the first part of the paths
        page_list.append(pages[i][0])
    # create a new column with the first part of the path
    traffic['lesson'] = page_list
    # return the prepared dataframe
    return traffic

In [23]:
def wrangle_web_traffic():
    '''
    this will perform both acquire and preparation steps in one command
    '''
    # acquire and prepare data
    return prepare_web_traffic(acquire_web_traffic())

In [2]:
traffic = pd.read_csv('anonymized-curriculum-access.txt', sep=' ', header=None)

In [3]:
traffic.head()

,0,1,2,3,4,5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


### This data is supposed to contain cohort and student info along with machine
- There are 981 unique entries in column 3 and 47 entries in column 4
- I'm thinking that column 3 is student, and column 4 is cohort
- column 5 is clearly ip

In [4]:
traffic = traffic.rename(columns={
    0:'date', 1:'time', 2:'page', 3:'user_id', 4:'cohort', 5:'ip'})

In [5]:
traffic['datetime'] = traffic.date + ' ' + traffic.time

In [6]:
traffic.datetime = pd.to_datetime(traffic.datetime)

In [7]:
traffic = traffic.drop(columns=['date', 'time']).set_index('datetime')

In [8]:
pages = traffic.page.str.split('/')

In [9]:
pages[7][0]

'javascript-i'

In [10]:
len(pages)

900223

In [11]:
range(len(pages))

range(0, 900223)

In [12]:
# for i in range(len(pages)):
#     print(f'{i}')

In [13]:
page_list=[]

In [14]:
page_list

[]

In [15]:
traffic.page.str.split('/')

datetime
2018-01-26 09:55:03                                                 [, ]
2018-01-26 09:56:02                                            [java-ii]
2018-01-26 09:56:05               [java-ii, object-oriented-programming]
2018-01-26 09:56:06                [slides, object_oriented_programming]
2018-01-26 09:56:24                         [javascript-i, conditionals]
                                             ...                        
2021-04-21 16:41:51                              [jquery, personal-site]
2021-04-21 16:42:02                                 [jquery, mapbox-api]
2021-04-21 16:42:09                          [jquery, ajax, weather-map]
2021-04-21 16:44:37    [anomaly-detection, discrete-probabilistic-met...
2021-04-21 16:44:39                                 [jquery, mapbox-api]
Name: page, Length: 900223, dtype: object

In [16]:
# for i in range(len(pages)):
#     print(f'{i}')

In [17]:
traffic = traffic.dropna()

In [18]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   page     847329 non-null  object 
 1   user_id  847329 non-null  int64  
 2   cohort   847329 non-null  float64
 3   ip       847329 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 32.3+ MB


In [19]:
page_list=[]
pages = traffic.page.str.split('/')
for i in range(len(pages)):
    page_list.append(pages[i][0])
traffic['lesson'] = page_list

In [1]:
traffic.head()

NameError: name 'traffic' is not defined

In [4]:
traf = acquire_web_traffic()
traf

opening file from local directory


,date,time,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2.0


In [25]:
traf = wrangle_web_traffic()
traf

opening file from local directory


,path,user_id,cohort_name,ip,name,start_date,end_date,program_id,lesson
datetime,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,java-ii
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,java-ii
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,slides
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2,javascript-i
...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2,jquery
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2,jquery
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2,jquery


In [8]:
traf = traf[traf.path.isna() == False]

In [26]:
traf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   path         847329 non-null  object 
 1   user_id      847329 non-null  int64  
 2   cohort_name  847329 non-null  float64
 3   ip           847329 non-null  object 
 4   name         847329 non-null  object 
 5   start_date   847329 non-null  object 
 6   end_date     847329 non-null  object 
 7   program_id   847329 non-null  int64  
 8   lesson       847329 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 64.6+ MB
